### 1\. **PCA on 3D dataset**

* Generate a dataset with 3 features each with N entries (N being ${\cal O}(1000)$). With $N(\mu,\sigma)$ the normali distribution with mean $\mu$ and $\sigma$  standard deviation, generate the 3 variables $x_{1,2,3}$ such that:
    * $x_1$ is distributed as $N(0,1)$
    * $x_2$ is distributed as $x_1+N(0,3)$
    * $x_3$ is given by $2x_1+x_2$
    

In [2]:
import numpy as np
import pandas as pd
N = 10**5
data1 = np.random.normal(loc=0.,scale=1.,size=N)
data2 = data1 + np.random.normal(loc=0.,scale=3.,size=N)
data3 = 2*data1 + data2
df = pd.DataFrame(data=np.array([data1,data2,data3]),index=['x1','x2','x3'])

* Find the eigenvectors and eigenvalues of the covariance matrix of the dataset

In [3]:
from scipy import linalg as la
co_mat=np.cov(df)
e_val,e_vect=la.eig(co_mat)
print(e_val,e_vect)

[ 2.70301787e+01+0.j -5.48432680e-15+0.j  2.00819322e+00+0.j] [[-0.11516096 -0.81649658  0.56574843]
 [-0.57773653 -0.40824829 -0.70679123]
 [-0.80805845  0.40824829  0.42470563]]


* Find the eigenvectors and eigenvalues using SVD. Check that the two procedures yield to same result

In [4]:
# I use SVD
e_vect_svd, spectrum, Vt = np.linalg.svd(co_mat)
# and estimate again that the eigenvalues and eigenvectors
e_val_svd = spectrum**2/(N-1)
# to check if the results from the two procedures are the same
print(e_val_svd,e_vect_svd)

[7.30637869e-03 4.03288032e-05 3.28459651e-34] [[-0.11516096  0.56574843 -0.81649658]
 [-0.57773653 -0.70679123 -0.40824829]
 [-0.80805845  0.42470563  0.40824829]]


In [82]:
# despite a switch in the column positions, the results are the visually the same

In [6]:
# we notice that one od the eigenvalues is extremely small: this is a sign that, through PCA, it will be possible to
# reduce the dimensionality of the problem by, at least, one 

* What percent of the total dataset's variability is explained by the principal components? Given how the dataset was constructed, do these make sense? Reduce the dimensionality of the system so that at least 99% of the total variability is retained.

In [79]:
# to find how much of the dataset variability is explained by the principal components
print('percentuali: ',e_val_svd/e_val_svd.sum()*100)

percentuali:  [9.94475452e+01 5.52454811e-01 4.68378762e-30]


In [7]:
# we see that
# - most of the variability (over 99%) lies in x1
# - a small amount (0.8%) of variability is due to x2
# - an insignificant amount of variability is related to x3: as by definition, x3 is strictly correlated to the other variables
#   x1 and x2 and can be easily dropped in the analysis
# furthermore, the goal is to include at least 99% of the system, which is well describe through x1: also x2 can be dropped
# we're left with just
print(data1)

[-0.35685963 -0.69092639  1.28779555 ...  0.86664872 -0.99047314
 -1.29982575]


* Redefine the data in the basis yielded by the PCA procedure

In [9]:
# to get the data in the eigenvector basis
e_df = pd.DataFrame(np.dot(e_vect.T,df))
print(e_df)

          0             1             2             3             4      \
0  1.576866e+00 -6.086475e+00 -4.134244e+00  2.800460e-01  5.990158e+00   
1 -5.635634e-16 -4.867738e-15  2.514698e-15  2.089218e-15  3.116993e-15   
2 -3.097953e-01 -2.460194e+00  1.434723e+00  1.092518e+00  1.447434e+00   

          5             6             7             8             9      ...  \
0 -1.078027e+00  4.374194e-01 -1.243904e+00 -2.093829e+00 -7.764924e-01  ...   
1 -3.637312e-15 -2.001867e-16 -5.655395e-16  1.195792e-15 -6.743954e-15  ...   
2 -1.883132e+00 -1.221996e-01 -2.801058e-01  6.274203e-01 -3.488683e+00  ...   

          99990         99991         99992         99993         99994  \
0 -7.122696e-01  6.906407e+00  1.512762e+00  4.044138e+00 -5.659210e+00   
1 -4.223381e-15 -1.251479e-15  4.004910e-15  1.806086e-15  1.013292e-15   
2 -2.196384e+00 -7.460391e-01  2.125418e+00  9.231789e-01  6.237048e-01   

          99995         99996         99997         99998         99999  
0  

* Plot the data points in the original and the new coordiantes as a set of scatter plots. Your final figure should have 2 rows of 3 plots each, where the columns show the (0,1), (0,2) and (1,2) proejctions.

### 2\. **PCA on a nD dataset**

Start from the dataset you have genereted in the previous exercise and add uncorrelated random noise. Such noise should be represented by other 10 uncorrelated variables normal distributed, with standar deviation much smaller (say, a factor 50) than those used to generate the $x_1$ and $x_2$.

Repeat the PCA procedure and compare the results with what you obtained before

# 8.3 \. **Looking at an oscillating spring** (60 MINUTES)

Imagine you have $n$ cameras looking at a spring oscillating along the $x$ axis. Each  camera record the motion of the spring looking at it along a given direction defined by the pair $(\theta_i, \phi_i)$, the angles in spherical coordinates. 

Start from the simulation of the records (say ${\cal O}(1000)$) of the spring's motion along the x axis, assuming a little random noise affects the measurements along the $y$. Rotate such dataset to emulate the records of each camera.

Perform a Principal Component Analysis on the thus obtained dataset, aiming at finding the only one coordinate that really matters.


In [53]:
import scipy.linalg as la

cameras = 5
theta =  np.random.uniform(0,90,cameras)
phi = np.random.uniform(0,180,cameras)
time = 100
r = np.random.uniform(0,1,time)
meas = np.zeros((cameras,time))
for i in range(cameras):
    meas[i,:] = r*np.sin(phi[i])*np.cos(theta[i])
val, vect = la.eig(np.cov(meas))
print(np.abs(val)/la.norm(val)*100)

[1.66710003e-14 1.00000000e+02 1.00079412e-15 1.38752551e-14
 6.41470121e-15]


### 4\. **PCA on the MAGIC dataset** (optional)

Perform a PCA on the magic04.data dataset

In [ ]:
# get the dataset and its description on the proper data directory
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/magic/magic04.data -P ~/data/
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/magic/magic04.names -P ~/data/ 